**Author:** Shahab Fatemi

**Email:** shahab.fatemi@umu.se   ;   shahab.fatemi@amitiscode.com

**Created:** 2024-04-19

**Last update:** 2025-09-07

**MIT License** — Shahab Fatemi (2025); For use in the *Machine Learning in Physics* course, Umeå University, Sweden; See the full license text in the parent folder.

<hr>

📢 <span style="color:red"><strong> Note for Students:</strong></span>

* Before working on the labs, review your lecture notes.

* Please read all sections, code blocks, and comments **carefully** to fully understand the material. Throughout the labs, my instructions are provided to you in written form, guiding you through the materials step-by-step.

* All concepts covered in this lab are part of the course and may be included in the final exam.

* I strongly encourage you to work in pairs and discuss your findings, observations, and reasoning with each other.

* If something is unclear, don't hesitate to ask.

* Exercise submission is not required; these tasks are designed to help you practice, explore the concepts, and learn by doing.

* I have done my best to make the lab files as bug-free (and error-free) as possible, but remember: *there is no such thing as bug-free code.* If you observed any bugs, errors, typos, or other issues, I would greatly appreciate it if you report them to me by email. Verbal notifications are not work, as I will likely forget 🙂

ENJOY WORKING ON THIS LAB.
***

# 🛠️ Purpose and Learning Outcomes:

In the previous lab, you (we) worked on a linear function, $h(x)=w_0 + w^Tx$. In this lab, we move to a high-degree polynomail function and we want to apply the OLS (closed-form) solution to our non-linear problems.

You will learn about:
* feature augmentation
* feature normalization
* workflow pipelines
* data splitting
* model training
* cross validation

I modified this lab from its original notebook to make it ready as soon as possible for your lab session. In case, something was confusing or inconsistent, please report it to me.

***

First, **we need data** to train our model.

We simulate the motion of a projectile by calculating the true displacement over time using the equation of motion, $y(t) = -(1/2)gt^2 + v_0t + y_0$. To mimic the real world measurement error, we add gaussian noise to the data using `np.random.normal` function. The aim is to fit a line into the generated data, which will be conducted in the next sections.

Let's generate and visualize data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

"""
    Creating a list of colors based on the "tab10" colormap.
    I want to use the color set in the "tab10" colormap for my plotting.
"""
cmap = plt.colormaps["tab10"]
colors = [cmap(i) for i in range(21)]

In [ ]:
#################################################################
# Simulate the motion of a projectile
# Compute the true y values for the motion of a projectile
def projectile_motion(g, v0, y0, time_range=(0, 5), num_points=100, noise_level=5, seed=42):
    np.random.seed(seed)  # Set random seed for reproducibility
    
    # time for the projectile motsion
    t = np.linspace(time_range[0], time_range[1], num_points)
 
    # The projectile equation is y = -(1/2)*g*t^2 + v0*t + y0
    y_true  = -0.5*g*t**2 + v0*t + y0 # true trajectory
    y_noisy = y_true + np.random.normal(0, noise_level, num_points) # noisy data
    return t, y_true, y_noisy

# Plot the projectile motion
# NOTE: This function visualizes the projectile motion and its noisy observations.
# However, I've also included an option to visualize the fitted polynomial regression.
# If the fitting data is provided, the function will plot the regression curve as well;
# otherwise, it will only show the projectile motion and noisy observations.
# I've done this because I want to call this function again later.
def plot_data(t, y_true, y_noisy, x_fit=None, y_fit=None, poly_degree=None, title="Projectile Motion"):
    # Use high dpi for high-quality output. 
    #  Usually any dpi between 200 and 300 is good for publications.
    #  Never use dpi<=100. The default for dpi in plt.figure() is 100.
    # My recommendation is dpi=300, especially for reports and publications, 
    # however, dpi=300 may create larger file sizes, and large figures on your screen, 
    # but you can try ...!
    plt.figure(figsize=(6, 4), dpi=200)

    # Plot the noisy data
    # Use scatter plot to visualize data distribution, and remember to use alpha<1.0 for transparency.
    # If the reason for alpha<1 is not clear for you, ask me or discuss it with your friends.
    plt.scatter(t, y_noisy, color=colors[1], s=40, edgecolors="k", alpha=0.6, label=f"Noisy data")

    # Plot the true projectile motion curve
    if (y_true is not None):
        plt.scatter(t, y_true, color=colors[0], s=5, label="True y")

    # Plot the polynomial regression fit if provided
    if (x_fit is not None and 
        y_fit is not None):
        label_text = f"Polynomial degree {poly_degree}" if poly_degree is not None else "Regression fit"
        plt.scatter(x_fit, y_fit, color=colors[2], s=2, label=label_text)

    # Customize plot appearance
    plt.xlabel("Time (s)", fontsize=14)
    plt.ylabel("Displacement (m)", fontsize=14)
    plt.title(title, fontsize=16)
    plt.legend()
    plt.grid(True, linestyle="--", color="grey", linewidth=0.5, alpha=0.6)
    plt.tight_layout() # Fit all elements within the figure area
    plt.show()

# ======== MAIN ========
# Parameters for the projectile motion simulation
v0 = 25.0  # initial velocity (m/s)
g  = 9.8   # gravity acceleration (m/s^2)
y0 = 0.0   # initial position/height/displacement (m)
n  = 100   # number of points (#)
time_range  = (0, 5)  # time range from 0 to 5 seconds
noise_level = 5       # standard deviation of the noise

# Construct data
t, y_true, y_noisy = projectile_motion(g, v0, y0, time_range, n, noise_level)

# Visualize the noisy and true trajectoory of the projectile.
plot_data(t, y_true, y_noisy)


***
## 🤓 Tips: Polish final figures

Many of you will work on data analysis or even get a job as a data analyist after graduation. Here is my suggestion (recommendation) to you based on my personal experience: When you analyze data, your figures are the first thing people notice (of course the quality of the analysis are above that). So make the figures and final products clear and easy to read. You spend a lot of time on analyzing the data, so why not spending some time on final touch-ups like choosing good colors, labeling axes properly, adding a legend, and making sure your plots are not too crowded or too blured. These small improvements can make a big difference in how others understand and appreciate your work. It is a good habit to always check your figures before using them in a report or presentation. Ask yourself: Is everything readable? Does the figure tell the story I want it to tell? But don't overdo it. Keep it simple, consistent, and clean. A well-made figure can often say a lot more than a full paragraph.
***

# Basis Function (Polynomial) Regression

**Overview:** Basis functions transform the original input features into a new set of features to capture complex relationships in the data. Here, we want to use $\phi_j(x)=x^j$ (see lecture notes).

Now we perform polynomial regression to fit a line into the (assuming) "measured" data. We use the closed-form solution (OLS: Ordinary Least Squares estimator) to calculate the coefficients of the polynomial. We construct the `Vandermonde` matrix for the input values ${x}$ based on the specified polynomial degree. Then,, we compute the weight vector ${w}$ using the OLS formula: 

$w = (x^T x)^{-1} x^T y$, 

where, from now on, ${x}$ is the Vandermonde matrix and ${y}$ is the output noisy displacement data.

In [ ]:
# Fit (train) polynomial using closed-form solution
def fit_polynomial_OLS(x, y, degree):
    # Form the Vandermonde matrix for x
    X = np.vander(x, degree + 1, increasing=True)
    
    # Closed-form solution: w = (X^T * X)^(-1) * X^T * y
    # The @ operator is matrix multiplication, equivalent to np.matmul()
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    return w

# Predict polynomial values given coefficients
def predict_polynomial_OLS(x, coeffs):
    return np.polyval(coeffs[::-1], x)

poly_degree = 5  # degree of the polynomial function

# Regression on noisy data for polynomial degree of poly_degree
w = fit_polynomial_OLS(t, y_noisy, poly_degree)

# print coefficients/parameters of the model
print("\nPredicted Coefficients w:")
for i, coef in enumerate(w):
    print(f"w{i} = {coef:+.2f}")

# Generate new t values to predict its y values (y^*) using our trained model
t_vals = np.linspace(time_range[0], time_range[1], 3*n)  # This is new input values
y_pred = predict_polynomial_OLS(t_vals, w)   # This is y^* in the lecture notes

plot_data(t, y_true, y_noisy, x_fit=t_vals, y_fit=y_pred, poly_degree=poly_degree)

***
### 💡 Reflect and Run

- Check the shape of matrix ${X}$ formed by the Vandermonde in `fit_polynomial_OLS` and compare its share with your input matrix, `{t}`.

- Try experimenting with different polynomial degrees (e.g., 1, 2, 7, or 10) to see how the model changes. A low-degree polynomial may *underfit* the data and a very high-degree polynomial may *overfit* and capture noise and memorize all patterns. Observing this behavior is a great way to understand the bias-variance tradeoff.

- Compare the coefficients you got from different polynomial degrees with the true coefficients of your function ${y(t)}$.

***

### 🤓 Why 42?

Did you notice I used "42" in setting my random seeds?
You see this "42" everywhere when we set our seeds to a specific number for data reproducebility. The number comes from the novel "The Hitchhiker's Guide to the Galaxy" By Douglas Adams in 1979. "Throughout all versions, the series follows the adventures of Arthur Dent, a hapless Englishman, following the destruction of the Earth by the Vogons (a race of unpleasant and bureaucratic aliens) to make way for a hyperspace bypass. In their travels, Arthur comes to learn that the Earth was actually a giant supercomputer, created by another supercomputer, `Deep Thought`. Deep Thought had been built by its creators to give the answer to the *'Ultimate Question of Life, the Universe, and Everything'*, which, after eons of calculations, was given simply as `42`." [Ref: Wikipedia]

***

## Normalization, and Pipeline with Sklearn

In the this section, we use "SciKitLearn" (sklearn) pre-developed functions instead of our self-developed OLS form. In this section, you are going to learn new practical aspects of ML: i.e., 

   - data normalization and 
   - forming pipelines.

The code below "fits" a polynomial regression model of a specified degree to our noisy data of the projectile, and normalizes the input features for improved numerical stability and better model performance. The normalized parameters allow scaling of the input data (t) to have a mean of 0 and a standard deviation of 1 using `StandardScaler`. This process is called **feature scaling** or **Standardization** or **Normalization**, which is important for ML algorithms that are sensitive to the scale of input data.

Read more: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

I've made the normalization as an optional setting, so one can choose to use or not use it.

Additionally, we use a SciKit-Learn `pipeline` to streamline the process by combining normalization (if enabled), polynomial feature generation, and linear regression into a single, efficient workflow. 

Read more: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

Notes: 
1. The pipeline is an important concept as it combines data transformation and model fitting in a single step. This ensures a streamlined workflow, and therefore it minimizes errors in a developed code. It also helps you to provide a code clarity which is essential for consistency in ML applications where preprocessing and modeling need to be efficiently managed and tested.

2. In the code belew, we use `x.reshape(-1,1)` to convert a 1D array into 2D because sk-learn takes input features in the shape of $n \times p$: (n_samples, p_features). To understand how it works, you can test this in a separate code section:
```python
    a = np.array([1, 2, 3])
    print(a)
    print(a.reshape(-1, 1))
    print(a.reshape(1, -1))
```

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression

# Fit (train) polynomial regression model using sklearn pipeline
def fit_polynomial_sklearn(x, y, degree=2, normalize=True):
    if(normalize):
        # Create a pipeline with normalization
        model = make_pipeline(StandardScaler(), 
                              PolynomialFeatures(degree), 
                              LinearRegression())
    else:
        # Create a pipeline without normalization
        model = make_pipeline(PolynomialFeatures(degree), 
                              LinearRegression())

    # We need to use x.reshape, as required by sklearn
    model.fit(x.reshape(-1, 1), y)
    return model

# Predict polynomial values at new input data using the trained model
def predict_polynomial_sklearn(model, x_vals):
    return model.predict(x_vals.reshape(-1, 1))

# Print the coefficients of the trained sklearn model from the pipeline
def print_model_coefficients(model):
    lr = model.named_steps['linearregression']
    coef = lr.coef_
    intercept = lr.intercept_

    print(f"Intercept: {intercept:.4f}")
    for i, c in enumerate(coef):
        print(f"w_{i}: {c:.4f}")
        
def plot_multiple_fits(t, y_true, y_noisy, degrees, normalize=True, title="Polynomial Regression"):
    plt.figure(figsize=(6, 4), dpi=200)

    # Plot noisy data
    plt.scatter(t, y_noisy, color=colors[1], s=40, edgecolors="k", alpha=0.6, label="Noisy data")

    # Plot true function
    plt.scatter(t, y_true, color=colors[0], s=5, label="True y")

    # Generate dense grid for smooth predictions
    t_vals = np.linspace(min(t), max(t), 3*t.size)

    # Plot polynomial fits for each degree
    for idx, deg in enumerate(degrees):
        model  = fit_polynomial_sklearn(t, y_noisy, degree=deg, normalize=normalize)
        y_pred = predict_polynomial_sklearn(model, t_vals)
        color  = colors[idx + 1]  # offset by 1 since colors[0] used for true y
        plt.plot(t_vals, y_pred, color=color, linewidth=2, label=f"Polynom d={deg}")

    plt.xlabel("Time (s)", fontsize=14)
    plt.ylabel("Displacement (m)", fontsize=14)
    plt.title(title, fontsize=16)
    plt.legend()
    plt.grid(True, linestyle="--", color="grey", linewidth=0.5, alpha=0.6)    
    plt.tight_layout()
    plt.show()

# ========== MAIN ==========
# Fit polynomial regression models and plot
degrees = [1, 5, 15] # degrees of the polynomial
plot_multiple_fits(t, y_true, y_noisy, degrees, normalize=True)

***
### 💡 Reflect and Run

- Modify the code to print the polynomial coefficients (weights) computed by the sklearn model for each polynomial degree. Then compare them with the coefficients you obtained earlier using the closed-form OLS solution. Explore the similarities and differences of the results. Observe how close the coefficients are, especially for lower-degree polynomials, and think about why they might differ more as the degree increases.

- Investigate the role of normalization in the pipeline by switching the `normalize` ON and OFF in the `plot_multiple_fits` function. Try this for a few polynomial degrees and observe the changes in the fitted curves and coefficients. Does normalization help the model perform better in the presence of noise, and did the shape of the fit is noticeably affected? Perhaps only focus on the coefficients made by the polmynoial degree 5.

- You can also increase the noise level in the data and examine your findings on a new more noisy data. You can increase noise level by increading `noise_level` from 5 to e.g., 10 in the very first code section in this notebook. Discuss your conclusions with your peers. 

***

# Overfitting

I want you to re-produce data overfitting through lowering the number of samples. The code below procudes that for you. Note that I've decrease sample points to n=12 in the code below. We see an "extreme" overfitting when the number of data (n) is equal to or smaller than the number of polynomial degrees.

In [ ]:
def overfit_example():
    # Parameters for the projectile motion simulation
    v0 = 25.0         # initial velocity (m/s)
    g  = 9.8          # gravity acceleration (m/s^2)
    y0 = 0.0          # initial position/height/displacement (m)
    n_samples = 12    # number of points (#)
    noise_lvl = 5     # standard deviation of the noise level
        
    # Generate data with a quadratic form and some noise
    t, y_true, y_noisy = projectile_motion(g, v0, y0, num_points=n_samples, noise_level=noise_lvl, seed=42)

    # Define degrees of polynomials to fit
    degrees = [2, 7, 12]
    plot_multiple_fits(t, y_true, y_noisy, degrees)

# ========== MAIN ==========
# Run the overfit example
overfit_example()

***
### 💡 Reflect and Run
- Set the polynomial degrees to `degrees = [1, 3, 13]` and run the code. Observe how each model fits the data, especially the one with degree "13", which is higher than the number of data points. What does it tells you about overfitting in polynomial regression? You can also test higher degrees, e.g., 22.

- Finally, return to `degrees = [1, 3, 13]`, but now significantly increase the number of data points to `n = 1000` (or higher, e.g., 10000). Rerun the code and examine how each model responds to this denser dataset. Pay particular attention to the high degree polynomials and discuss on how increasing the amount of data affects model flexibility and the ability to **generalize**.

***

# Optimal Polynomial Degree

Let's generate a new data using a high order polynomials. We also separate our data into training and validation sets. For data splitting, we sue sklearn's `train_test_split`, assuming 70% of the data is used for training and 30% for validation. As I said, the "validation" and "test" terms are used interchangably. Since the function is named `train_test_split`, we also call our sets train and "test", but we know that the "test" set is for validation.

In [ ]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# Data generation function
def generate_new_data(a=+1, b=-5, c=+3, x_range=(-2, 2), 
                      num_points=100, noise_level=3.0, seed=42):
    np.random.seed(seed)
    x = np.linspace(x_range[0], x_range[1], num_points)
    y_true = a * x**5 + b * x**3 + c * x
    y_noisy = y_true + np.random.normal(0, noise_level, num_points)
    return x, y_true, y_noisy

# ========== MAIN ==========
# Parameters
a = +1
b = -7
c = +3
n = 100
x_range = (-2.5, 2.5)
noise_level = 7.0

# Generate data
x, y_true, y_noisy = generate_new_data(a, b, c, x_range, n, noise_level)

# Randomly split data (both true and noisy) into training and test sets
x_train, x_test, y_train_noisy, y_test_noisy, y_train_true, y_test_true = train_test_split(
    x, y_noisy, y_true, test_size=0.3, random_state=42)

# Plot only the training set
plot_data(x_train, y_true=y_train_true, y_noisy=y_train_noisy, title="Training Data")

We are given the dataset above that follows a non-linear function, and our task is to fit a polynomial regression model to the data. One of the key challenges in this process is determining the appropriate degree "d" for the polynomial. A degree that is too low may result in underfitting (simple model), and a degree that is too high can lead to overfitting (complex model). We want neither of them. 

The goal is to identify the "optimal" degree that balances model complexity and **generalization**. In this section, we want to explore how different values of "d" (or degrees) affect the results obtained from our model. In our example, we test degrees = [2, 3, 5, 8]. The question is, which of these degrees is most optimal value? Let's find out!

First, we plot our predictions obtained from our regression model for different polynomial degrees.

In [ ]:
degrees = [2, 3, 5, 8] # degrees of the polynomial
plot_multiple_fits(x_train, y_train_true, y_train_noisy, degrees)

To address the issue discussed above, we assess the performance of the regression models across various polynomial degrees by computing three evaluation metrics: MSE, RMSE, and MAE score. You can also include R2 score if you want. For each polynomial degree, we use the trained model to make predictions, calculate the metrics, and plot the results to visualize how model accuracy changes with increasing model's complexity. Remember that `prediction` should be applied on the validation/test data.

For those of you not familiar with Python: Skip the code in the function, and read the lines of code after the function. Then move up and study the function. If complicated to trace, you're welcome to ASK!

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

# Function to validate models and plot the results
# This functions evaluates the performance of polynomial regression models.
# It computes metrics like MSE, MAE, and RMSE for each model.
# Its input includes the feature matrix (x), true target values (y_true),
# a list of trained models, and their corresponding polynomial degrees.
def validate_poly_models(x, y_true, models, degrees):
    mse_values  = []
    mae_values  = []
    rmse_values = []

    for model in models:
        # Predict y values using the model
        y_pred = model.predict(x[:, np.newaxis])

        mse  = mean_squared_error(y_true, y_pred)       # Calculate MSE
        mae  = mean_absolute_error(y_true, y_pred)       # Calculate MAE
        rmse = root_mean_squared_error(y_true, y_pred)   # Calculate RMSE

        # Store the metrics for plotting
        mse_values.append(mse)
        mae_values.append(mae)
        rmse_values.append(rmse)

    plt.figure(figsize=(5, 4), dpi=300)

    # Plot metrics
    # Note: For better visualization, I use a logarithmic scale for the y-axis.
    plt.semilogy(degrees, mse_values , marker="o", markersize=5, color=colors[0], linestyle="-" , label="MSE" )
    plt.semilogy(degrees, mae_values , marker="^", markersize=5, color=colors[1], linestyle="-" , label="MAE" )
    plt.semilogy(degrees, rmse_values, marker="s", markersize=5, color=colors[2], linestyle="--", label="RMSE")

    # Set x-ticks
    plt.xticks(degrees)

    # Plot
    plt.xlabel("Polynomial Degree", fontsize=14)
    plt.ylabel("Metric Value", fontsize=14)
    plt.title("Model Validation Metrics", fontsize=14, weight="bold") # Here I show how to bold the title
    plt.legend(fontsize=10, title_fontsize=12)
    plt.grid(True, linestyle="--", color="grey", linewidth=0.5, alpha=0.6)
    plt.tight_layout()
    plt.show()

# ========== MAIN ==========
# degrees of the polynomial
# We give these as input to our training model
degrees = np.linspace(1, 12, 12, dtype=int)

# We train data with different polynomial degrees and return the models into 
#  an array for use in the validation function.
models  = [fit_polynomial_sklearn(x_train, y_train_noisy, degree) for degree in degrees]

# Validate the models and show the metrics
validate_poly_models(x_test, y_test_noisy, models, degrees)

***
### ✅ Check your understanding

- Study the figure above. Which of the polynomial degrees is the most optimal value, and why?

- Why does the metric value increase after a certain polynomial degree?

- How much does the noise level and number of sample points affect the results? Change only one of the at a time, re-run the code and observe associated changes.

### ⛷️ Exercise (Do it yourself!)
Write a code that trains polynomial regression models of varying degrees and computes the RMSE for both the training and validation/test sets. Plot the RMSE values as a function of polynomial degree and identify the generalization error.

***

## Bias-Variance Decomposition

In order to run this section, you need to install an extra package `mlextend`:

If you use pip3:
```bash
    $ pip3 install mlxtend
```

If you use pip:
```bash
    $ pip install mlxtend
```

If you use conda:
```bash
    $ conda install conda-forge::mlxtend
```


**overview** 
The bias-variance tradeoff shows the balance between two sources of error that can affect the performance of predictive models: bias and variance.

#### Bias 
It measures how far a model's average predictions are from the true values.
* High bias: model is too simple (e.g. underfitting)
* Low bias: model can capture the patterns well

#### Variance:
It measures how much the model's predictions vary if we train it on different datasets. The variance between models.
* High variance: model is too sensitive to noise and outliers (e.g. overfitting)
* Low variance: model is stable on different training sets

#### Total MSE error
MSE = $\text{Bias}^2$ + Variance + irreducible noise


In [ ]:
from mlxtend.evaluate import bias_variance_decomp

# ========== MAIN ==========
# Parameters
a = +1
b = -7
c = +3
n = 500
x_range = (-2.5, 2.5)
noise_level = 4.0

# Generate data
x, y_true, y_noisy = generate_new_data(a, b, c, x_range, n, noise_level)

# Randomly split data (both true and noisy) into training and test sets
x_train, x_test, y_train_noisy, y_test_noisy, y_train_true, y_test_true = train_test_split(
    x, y_noisy, y_true, test_size=0.3, random_state=42)

degrees = np.arange(1, 15, dtype=int)
models  = [fit_polynomial_sklearn(x_train, y_train_noisy, degree) for degree in degrees]

mse_list   = []
bias2_list = []
var_list   = []

# Loop over models
for model in models:
    mse, bias2, var = bias_variance_decomp(
        model,
        X_train=x_train.reshape(-1, 1),
        y_train=y_train_noisy,
        X_test=x_test.reshape(-1, 1),
        y_test=y_test_noisy,
        loss='mse', num_rounds=20, random_seed=42)
    
    mse_list.append(mse)
    bias2_list.append(bias2)
    var_list.append(var)

# Plot bias-variance trade-off
plt.figure(figsize=(8, 5), dpi=200)
plt.semilogy(degrees, mse_list, label='Total MSE Error', marker='o')
plt.semilogy(degrees, bias2_list, label=r'$Bias^2$', marker='s')
plt.semilogy(degrees, var_list, label='Variance', marker='^')

plt.xlabel("Polynomial Degree", fontsize=14)
plt.ylabel("Error", fontsize=14)
plt.title("Bias-Variance Trade-off", fontsize=16)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


***
### ✅ Check your understanding
- What do you observe from the bias-variance decomposition plot?

- How does the trend in $bias^2$, variance, and total error help you identify the optimal polynomial degree for your model?

### 💡 Reflect and Run
- Try changing the number of training samples, the noise level, and the true polynomial degree of the data. Vary one factor at a time, then explore combinations of changes. Observe how each change affects bias-variance, and the model's generalization performance.

- You can change the parameters `(a, b, c)` for your data generator function `generate_new_data` and re-run all the analysis above for your new data. Additionally, if you want, you can create your own data generator and work on your own data. For example, I suggest generating data using this function:
```python
        def polysine(x, 
                    poly_coeffs=[2, -3, 0, 1], 
                    sine_amp=3.0, 
                    sine_freq=4.0, 
                    noise_level=1.0, 
                    seed=42):

            # Polynomial part
            poly = np.polyval(poly_coeffs, x)

            # Sine-wave part
            sine = sine_amp * np.sin(sine_freq * x)

            # Noise
            np.random.seed(seed)
            noise = np.random.normal(0, noise_level, size=x.shape)

            return poly + sine + noise
```

Do you understand how the data is generated using the function above? Can you write the $f(x)$?

***

# Cross-validation

Imagine you have a set of experimental data, and you want to build a model to predict outcomes based on that data. You could just use all your data to train your model, but that would not tell you how well it will work on new data. Cross Validation is like running multiple experiments with different conditions to see how well your model works.

Let's implement a k-fold cross validation to evaluate the performance of our regression model for various polynomial degrees. The code below splits the dataset into k subsets (folds), shuffling the data each time to ensure randomness. For each fold, it trains the model on the training set and evaluates it on the validation set, calculating the RMSE for each degree of the polynomial. The average RMSE for each polynomial degree is then computed across all folds, providing a measure of the model's **generalization** performance.

In [ ]:
from sklearn.model_selection import KFold

# This function performs k-fold cross-validation for a given model function (polynomial) 
# and a set of degrees. The inputs are the feature matrix (x), target values (y), 
# the model function, a list of polynomial degrees, and the number of folds (k).
def cross_validation_kfold(x, y, model_function, degrees, k):
    # KFold splitter. 
    # Shuffling the data is crucial here)!
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    results = { degree: [] for degree in degrees }  # Store RMSE for each degree

    # Iterate over each polynomial degree
    for degree in degrees:
        for train_idx, val_idx in kf.split(x):
            # Split data into training and validation
            x_train, x_val = x[train_idx], x[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            # Train the model on the current k-th fold of data and for the d-th degree polynomial
            model = model_function(x_train, y_train, degree)
            
            # Predict on validation set
            y_pred = model.predict(x_val.reshape(-1, 1))
            
            # Compute RMSE and store
            rmse = root_mean_squared_error(y_val, y_pred)
            results[degree].append(rmse)
    
    # Compute average RMSE for each degree
    avg_rmse = {degree: np.mean(rmses) for degree, rmses in results.items()}
    return avg_rmse

# Visualize cross-validation results
def plot_validation_results(degrees, avg_rmse):
    plt.figure(figsize=(6, 3), dpi=200)
    plt.plot(degrees, [avg_rmse[degree] for degree in degrees], 
             marker="o", linestyle="-", color=colors[0])
    plt.xlabel("Polynomial degrees", fontsize=12)
    plt.ylabel("Avg. RMSE", fontsize=12)
    plt.title("K-Fold Cross-Validation", fontsize=14)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

# ========== MAIN ==========
# Parameters
a = +1
b = -7
c = +3
n = 100
x_range = (-2.5, 2.5)
noise_level = 4.0

# Generate data
x, y_true, y_noisy = generate_new_data(a, b, c, x_range, n, noise_level)

# Perform k-fold cross-validation
avg_rmse = cross_validation_kfold(x, y_noisy, 
                                  model_function=fit_polynomial_sklearn, 
                                  degrees=degrees, 
                                  k=5)

# Print average RMSE for each degree
for degree, rmse in avg_rmse.items():
    print(f"Degree {degree}: Avg RMSE = {rmse:.4f}")

# Plot results
plot_validation_results(degrees, avg_rmse)

***
### ✅ Check your understanding

- After I generated the data in the code section above, why did not I use `train_test_split`? 

- For the same dataset (same number of sample, noise level, etc.) compare the RMSE results from the `validate_poly_models` function from what you get using `cross_validation_kfold`. Any differences in the RMSE curve? Why?


### 💡 Reflect and Run
I used KFold cross validation (CV) from sklearn. However, you know there are other methods. Apply other CV models in your code and compare their results. For this specific dataset, do you expect differences from different CV models?


***
# ⛷️ Exercise

The gravitational potential $\Phi(r, \theta)$ of a planet is a scalar quantity that describes the gravitational potential energy per unit mass at a point in space. It depends on the radial distance $r$ from the center of the plant and the polar angle $\theta$ (measured from the planet's axis of symmetry). 

The general form of the gravitational potential is:

$$
\Phi(r, \theta) = -\frac{\mu}{r} \left( 1 + \sum_{\ell=2}^{L} \frac{J_{\ell} R_p^\ell}{r^{\ell}} P_\ell(\cos(\theta)) \right),
$$

Where:
- $\mu = G M$ is the standard gravitational parmerter, where $G=6.6743 × 10^{-11}\, \text{m}^3 \text{kg}^{-1} \text{s}^{-2}$ is the gravitational constant and $M$ is the mass of the planet.
- $r$ is the radial distance from the center of the planet.
- $\theta$ is the polar angle, typically ranging from 0 (at the north pole) to $\pi$ (at the south pole).
- $R_p$ is the radius of the planet (often the equatorial radius).
- $J_{\ell}$ are the spherical harmonic coefficients that describe the higher-order gravitational moments of the planet, typically used to account for non-spherical mass distributions (such as flattening at the poles or bulging at the equator).
- $P_\ell(\cos(\theta))$ are the Legendre polynomials of degree $\ell$, which depend on the polar angle $\theta$ and are used to represent the multipole expansion of the gravitational potential.

### Explanation:

1. **Monopole Term**: The first part of the equation, $-\frac{\mu}{r}$, represents the gravitional potential for a perfectly spherical planet. This is the **monopole term** and describes the gravitational potential due to the central mass of the planet.
   
2. **Multipole Terms**: The second part of the equation, $\sum_{\ell=2}^{L} ...$, accounts for the non-spherical shape of the planet. It includes terms that describe the **higher-order gravitational moments**, which are important for planets that are not perfect spheres (e.g., planets with equatorial bulges or irregular shapes, which are very common in the solar system. e.g., Earth is not a perfect sphere). These terms are known as **multipole expansions**:
    - $\ell = 2$ corresponds to the **quadropole** moment (flattening of the planet).
    - $\ell = 3$ corresponds to the **octopole** moment, and so on. (I cannot envision the effects for higher terms.)
   
3. **Legendre Polynomials**: The Legendre polynomials, $P_\ell[\cos(\theta)]$, describe how the potential changes with respect to the polar angle $\theta$. They ensure that the gravitational potential is consistent with the spherical symmetry of the planet's mass distribution.

### Your tasks:

A spacecraft has provided measurements for the gravitational potential around a planet with a gravitational parameter $\mu = 5.793960 \times 10^{15} \, \text{m}^3 \, \text{kg}^{-1} \, \text{s}^{-2}$. The observed data for the gravitational potential is stored in the file `./datasets/gravitational_potential_data.csv`. 

#### Steps for Analysis:

- **Download and Visualize the Data**: The data includes measurements of the gravitational potential, with the units for each parameter being:
   - $r$ (radial distance) in **meters** (m),
   - $\theta$ (polar angle) in **radians** (rad),
   - $\Phi$ (gravitational potential) in **Joules per kilogram** (J/kg).

   The dataset contains some measurement noise.

- **Important step:** Before processiding any further, read about the Legendre polynomials from: https://en.wikipedia.org/wiki/Legendre_polynomials

- **Polynomial Regression**: To model the gravitational potential, we can use a polynomial regression. The goal is to determine the values of the spherical harmonic coefficients $J_2$ and $J_3$, which describe the higher-order gravitational moments. Since the spacecraft could not measure higher-order terms, we limit the expansion to $\ell = 3$, meaning the maximum order is $J_3$.

- **Analysis Goals**:
   - **Extract the values** of $J_2$ and $J_3$ from the data using polynomial regression.
   - This exercise will be continued in the next lab, using Regularization.

***
END
***